In [1]:
using JuMP
using Gurobi
using DelimitedFiles
using CSV

## File handling

In [2]:
#this function inputs multiple string arrays and outputs a matrix of int and float
function arrayToMatrix(array)
    h=[]
    counter=0
    for element in array
        counter=0
        g=split(element)
        b=[]
        #converting strings to int or float
        for item in g
            if occursin(".",item)
                push!(b,parse(Float64,item))
                continue
            end
            push!(b,parse(Int64,item))
            counter+=1
        end
        push!(h,b)
    end
    #creating an empty matrix and thereafter adding all the arrays in the matrix
    m = Matrix{Any}(nothing, 0, counter)
    for element in h
        m=[m;element']
    end
    return m
end
 
#this function inputs a textfile and outputs a dictionary
function readFromFile(text)
    a=readlines(text)
    dict=Dict([])
    key=""
    array=[]
    flag=false
    for element in a
        if occursin(";", element)
            ja=arrayToMatrix(array)
            push!(dict, key=>ja)
            flag=false
            array=[]
            continue
        end
        if flag==true
            push!(array, element)
        end
        if occursin("#",element)
            flag=true
            key=element[2:end]
        end
    end
    return dict
end
;

## Sets

In [3]:
I=10  # 10 pasientgrupper
U=2   # 1=hast og 2=ikke-hast
S=2   # 2 scannere
D=H=5 # 5 dager
R=13  # 13 radiologer
M=5   # 1=tolking intern, 2=tolking ekstern, 3=scanner, 4=MDT, 5=Åpen
;

## Parameters

In [4]:
fileName="BP_parameters.txt"

T_im=readFromFile(fileName)["T_im"]
G_dr=readFromFile(fileName)["G_dr"]
N_sd=readFromFile(fileName)["N_sd"]
S_sd=readFromFile(fileName)["S_sd"]
D_iuW=readFromFile(fileName)["D_iuW"]
K_dr=readFromFile(fileName)["K_dr"]
D_iuD=readFromFile(fileName)["D_iuD"]
B_iu=readFromFile(fileName)["B_iu"]
L_sd=readFromFile(fileName)["L_sd"]
A_id=readFromFile(fileName)["A_id"]
P=readFromFile(fileName)["P"][1]
Q_d=readFromFile(fileName)["Q_d"][1]
Q_w=readFromFile(fileName)["Q_w"][1]
E_iu1=readFromFile(fileName)["E_iu1"]
E_iu2=readFromFile(fileName)["E_iu2"]
E_iu3=readFromFile(fileName)["E_iu3"]
E_iu4=readFromFile(fileName)["E_iu4"]
E_iu5=readFromFile(fileName)["E_iu5"]
E_iud=cat(dims=3,E_iu1,E_iu2,E_iu3, E_iu4, E_iu5)
;


## Variables

In [5]:
m = Model(with_optimizer(Gurobi.Optimizer))
@variable(m, delta_p[1:D,1:R] >= 0, Int)
@variable(m, delta_m[1:D,1:R] >= 0, Int)
@variable(m, x[1:I,1:U,1:S,1:D] >= 0, Int)
@variable(m, w[1:I,1:U,1:D,1:R,1:M] >= 0, Int)
@variable(m, u_i[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, v[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, alpha[1:I,1:U,1:D], Bin)
@variable(m, beta[1:S,1:D,1:R], Bin)
@variable(m, gamma[1:I,1:D,1:R], Bin)
@variable(m, zeta[1:S,1:D]>=0, Int)#number of open slots at the scanner s at day d
;

Academic license - for non-commercial use only


## Objective function

In [6]:
@objective(m, Min, sum(delta_p[d,r]+delta_m[d,r] for d=1:D, r=1:R ));

## Constraints

In [7]:
#Constraint2 - Deltas
#Connects the delta variables to w
@constraint(m,[d=1:D, r=1:R], delta_m[d,r]-delta_p[d,r] + sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U, m=5)==G_dr[d,r])             
     
#Constraint3 - Slot balance
#Ensures that assigned slots+open slots=available slots-study slots
@constraint(m,[s=1:S,d=1:D], sum(x[i,u,s,d] for i=1:I,u=1:U)+zeta[s,d]==N_sd[s,d]-S_sd[s,d])
                            
#Constraint4 - Demand
#The number of slots must equal the demand
@constraint(m,[i=1:I,u=1:U], sum(x[i,u,s,d] for s=1:S,d=1:D)==D_iuW[i,u])

#Constraint5 - minHours per day
#minimum number of hours a radiologist can work per day
@constraint(m,[d=1:D,r=1:R], sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U,m=1:M)>=K_dr[d,r]-Q_d)

#Constraint6 - maxHours per day
#maximum number of hours a radiologist can work per day
@constraint(m,[d=1:D,r=1:R], sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U,m=1:M)<=K_dr[d,r]+Q_d)

#Constraint7
#The totalt number of working hours for each radiologist must equal Q_w
@constraint(m,[r=1:R], sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I,u=1:U, d=1:D, m=1:M)==Q_w)

#Constraint8
@constraint(m,[i=1:I,u=1:U], sum(alpha[i,u,d] for d=1:D)>=D_iuD[i,u])
#Constraint9
@constraint(m,[i=1:I,u=1:U, d=1:D], sum(x[i,u,s,d] for s=1:S)>=alpha[i,u,d])

#Constraint 10 - summen av alle tolkeoppgaver for u en gitt dag h må være lik antall tolkeoppgaver for w samme dag
@constraint(m,[i=1:I,u=1:U,h=1:H, r=1:R], sum(u_i[i,u,d,h,r] for d=1:D) == sum(w[i,u,h,r,m] for m=1))

#Constraint 11
@constraint(m,[i=1:I,u=1:U,h=1:H, r=1:R], sum(v[i,u,d,h,r] for d=1:D) == sum(w[i,u,h,r,m] for m=2))


#Constraint12                                                                                             
for i=1:I, u=1:U, d=1:D
    if d+B_iu[i] <= 7
        @constraint(m, [i,u,d], sum(x[i,u,s,d] for s=1:S) - sum(u_i[i,u,d,h,r] for r=1:R,h=d:d+B_iu[i] if h <= D) == 0)
    else
        @constraint(m, [i,u,d], sum(x[i,u,s,d] for s=1:S) - sum(u_i[i,u,d,h,r] for r=1:R,h=d:d+B_iu[i] if h <= D) - sum(u_i[i,u,d,h,r] for r=1:R, h=1:(d+B_iu[i])%7 if h<d) == 0)                                                                                                                      
    end                                                                                                 
end
                                    
#Constraint13                                                                                                                           
for i=1:I, u=1:U, d=1:D
    if d+B_iu[i,u] <= 7
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=d:d+B_iu[i,u] if h<=D) == E_iud[i,u,d])
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=1:d-1) == 0) #Setter starten av uka =0
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=(d+B_iu[i,u]+1:D) if h <= D)==0)  #20                                                                                                          
    else                                                                                                                  
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=d:d+B_iu[i,u] if h<=D) + sum(v[i,u,d,h,r] for r=1:R, h=1:(d+B_iu[i,u])%7 if h<d) == E_iud[i,u,d])
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R,h=((d+B_iu[i,u])%7) + 1:(d-1)) == 0)    #setter midten av uka =0                                                                                                                                    
    end                                                                                                 
end

#Constraint14 - Demand for radiologer på scannere på dag d
@constraint(m,[s=1:S,d=1:D], sum(beta[s,d,r] for r=1:R)==L_sd[s,d])

#Constraint15
@constraint(m, [r=1:R, d=1:D], sum(beta[s,d,r] for s=1:S)<=1)
                                                                                                
#Constraint16 - En radiolog kan ikke ha mer enn P scannetasks på en uke
@constraint(m,[r=1:R], sum(beta[s,d,r] for s=1:S, d=1:D)<=P) 
   
#Constraint17: For en gitt scanner en gitt dag for en gitt radiolog må 
#antallet scanneslots være like antall slots den dagen hvis radiologen er satt til scanning                                                                                      
@constraint(m,[d=1:D,r=1:R], sum(w[i,u,d,r,m] for i=1:I, u=1:U, m=3) == sum(beta[s,d,r] for s=1:S)) 

#Constraint18 NY
@constraint(m, [i=1:I,d=1:D], sum(w[i,u,d,r,m] for u=1:U, r=1:R, m=4)==A_id[i,d])

#Constraint19
@constraint(m,[i=1:I,d=1:D],sum(gamma[i,d,r] for r=1:R)==A_id[i,d])
                
#Constraint20
@constraint(m,[i=1:I,d=1:D, r=1:R], sum(w[i,u,d,r,m] for u=1:U, m=4)==gamma[i,d,r])               

#Constraint21 - Det må være like mange interne tolkeoppgaver som det er slots
@constraint(m, sum(w[i,u,d,r,m] for i=1:I, u=1:U, d=1:D, r=1:R, m=1)==sum(x[i,u,s,d] for i=1:I, u=1:U, s=1:S, d=1:D))                                                
;                                                                                                                                                                                                

## Results

In [ ]:
@time begin
    optimize!(m)
    println("Run time: ")
end

println("Status: ",termination_status(m))
println("Primal status: ",primal_status(m))
println("Dual status: ",dual_status(m))
#println("Dual con1: ", shadow_price(con1[1]))
println("Objective value: ", objective_value(m))

println()
println("****** Variable: x[i,u,s,d] ******")
for s=1:S
    println()
    println("Scanner: ",s)
    for d=1:D
        println()
        println("Day: ",d)
        println("__________________")
        for u=1:U
            for i=1:I
            if value(x[i,u,s,d]) != 0
                println("x[",i,",",u,",",s,",",d,"] = ", value(x[i,u,s,d]))
            end
            end
        end
        println("Open slots = ",value(zeta[s,d]))
    end
    
end
println()
println("****** Variable: w[i,u,d,r,m] ******")
for d=1:D
    println()
    println("Day: ",d)
    println("__________________")
    for r=1:R
        #println("Patient group: ", i)
        for i=1:I
            for u=1:U
                #println("Radioligst: ",r)
                for m=1:M
                    if value(w[i,u,d,r,m]) != 0
                        println("w[",i,",",u,",",d,",",r,",",m,"] = ", value(w[i,u,d,r,m]))
                    end
                end
            end
        end
    end
end
println()
println("****** Variable: u[i,u,d,h,r] ******")
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    if value(u_i[i,u,d,h,r]) != 0
                        println("u[",i,",",u,",",d,",",h,",",r,"] = ", value(u_i[i,u,d,h,r]))
                    end
                end
            end
        end
    end
end

println()
println("****** Variable: v[i,u,d,h,r] ******")
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    if value(v[i,u,d,h,r]) != 0
                        println("v[",i,",",u,",",d,",",h,",",r,"] = ", value(v[i,u,d,h,r]))
                    end
                end
            end
        end
    end
end

println()
println("****** Variable: delta[d,r] ******")
for d =1:D
    println()
    println("Day: ",d)
    println("__________________")
    for r=1:R
        if value(delta_p[d,r]) != 0
            println("delta_p[",d,",",r,"] = ",value(delta_p[d,r]))
        end
        if value(delta_m[d,r]) != 0
            println("delta_m[",d,",",r,"] = ",value(delta_m[d,r]))
        end
    end
end

println()
println("****** Variable: alpha[i,u,d] ******")
for i=1:I
    for u=1:U
        for d=1:D
            if value(alpha[i,u,d]) != 0
                println("alpha[",i,",",u,",",d,"] = ", value(alpha[i,u,d]))
            end
        end
    end
end

println()
println("****** Variable: beta[s,d,r] ******")
for s=1:S
    for d=1:D
        for r=1:R
            if value(beta[s,d,r]) != 0
                println("beta[",s,",",d,",",r,"] = ", value(beta[s,d,r]))
            end
        end
    end
end

println()
println("****** Variable: gamma[i,d,r] ******")
for i=1:I
    for d=1:D
        for r=1:R
            if value(gamma[i,d,r]) != 0
                println("gamma[",i,",",d,",",r,"] = ", value(gamma[i,d,r]))
            end
        end
    end
end

## The model

In [ ]:
#print(m)